# Test Backend simulatori instalation with monitor
### Dr. Oscar Montiel
#### Nov 27, 2024

In [ ]:
# Import required libraries
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Session
#from qiskit import QuantumCircuit, transpile

#from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, JobStatus
from qiskit import QuantumCircuit, transpile
import time
import logging

In [ ]:
# Step 1: Enable detailed logging for advanced monitoring
logging.basicConfig(level=logging.INFO)  # Enables INFO level logging for detailed updates

In [ ]:
# Step 2: Check if the API token is already saved
try:
    # Attempt to load the saved account
    service = QiskitRuntimeService()
    print("IBM Quantum account loaded successfully.")
except Exception:
    # If the account is not saved, prompt the user to save it
    print("IBM Quantum account not found. Saving the API token.")
    token = input("Enter your IBM Quantum API token: ")
    QiskitRuntimeService.save_account(channel="ibm_quantum", token=token, overwrite=True)
    service = QiskitRuntimeService()
    print("IBM Quantum account saved and loaded successfully.")

In [ ]:
# Step 3: List available backends
print("\nAvailable Backends:")
print(service.backends())

In [ ]:
# Step 4: Select a backend
backend_name = "ibm_brisbane"  # Replace with your desired backend
backend = service.backend(backend_name)  # Assign the backend using the name
print(f"\nSelected Backend: {backend_name}")

In [ ]:
# Step 5: Create a quantum circuit
qc = QuantumCircuit(1, 1)  # 1 qubit, 1 classical bit
qc.h(0)  # Apply a Hadamard gate to the qubit
qc.measure(0, 0)  # Measure the qubit into the classical bit

In [ ]:
# Print the circuit (optional visualization)
print("\nQuantum Circuit:")
print(qc)

In [ ]:
# Step 6: Transpile the circuit for the selected backend
compiled_circuit = transpile(qc, backend)
print("\nTranspiled Circuit:")
print(compiled_circuit)

In [ ]:
# Step 7: Use the Sampler primitive
sampler = Sampler(mode=backend)  # Initialize the Sampler with the backend

In [ ]:
# Step 8: Execute the circuit with monitoring
print("\nRunning the circuit...")
job = sampler.run([compiled_circuit], shots=1024)  # Specify the circuits and shots
print(f"Job ID: {job.job_id()}")

In [ ]:
# Monitor job status
print("\nMonitoring Job Status:")

while True:
    status = job.status()
    print(f"Current Status: {status}")
    
    if status == JobStatus.QUEUED:
        print(f"Position in Queue: {job.queue_position()}")
    elif status in [JobStatus.DONE, JobStatus.CANCELLED, JobStatus.ERROR]:
        print(f"Final Status: {status}")
        break
    
    time.sleep(10)  # Wait 10 seconds before checking again

# Fetch results once the job is complete
if job.status() == JobStatus.DONE:
    print("\nWaiting for results...")
    result = job.result()  # Get the results
    counts = result.quasi_dists[0]
    print("\nResults:")
    print("Quasi-Distribution Counts:", counts)

# Print additional job information
print("\nJob Details:")
print(f"Job ID: {job.job_id()}")
print("Backend:", job.backend())
print("Creation Date:", job.creation_date())
print("Final Status:", job.status())
